In [19]:
import helpers
import numpy as np
import re
import math
from tqdm import tqdm
import copy
import networkx as nx
from collections import Counter
import ast
import itertools as it
from shapely.geometry import Polygon

data = """#############
#...........#
###D#D#C#C###
  #B#A#B#A#
  #########"""

#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########

#with open("input_23.txt", 'r') as f:
#    data = f.read()

data = data.lstrip()
data = data.rstrip()
    
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = re.findall("(\w+) x=([-+]?\d+)..([-+]?\d+),y=([-+]?\d+)..([-+]?\d+),z=([-+]?\d+)..([-+]?\d+)", data)
#data = [x.rstrip() for x in data.split("\n\n") if x]
data


'#############\n#...........#\n###D#D#C#C###\n  #B#A#B#A#\n  #########'

In [20]:
move = {"A": 1, "B": 10, "C": 100, "D": 1000}
move_num = {1: 1, 2: 10, 3: 100, 4: 1000}
translation = {1: "A", 2: "B", 3: "C", 4: "D"}
exits = {0: 2, 1: 4, 2: 6, 3: 8}

In [40]:
hallway = np.zeros(11, dtype="int8")
hallway[[2,4,6,8]] = -1
#hallway[[5]] = 1
#hallway[[10]] = 3
print(hallway)
Aroom = np.array([4, 4, 4, 2])
Broom = np.array([4, 3, 2, 1])
Croom = np.array([3, 2, 1, 2])
Droom = np.array([3, 1, 3, 1])
#Aroom = np.array([2, 4, 4, 1])
#Broom = np.array([3, 3, 2, 4])
#Croom = np.array([2, 2, 1, 3])
#Droom = np.array([4, 1, 3, 1])
#Aroom = np.array([1, 1])
#Broom = np.array([2, 2])
#Croom = np.array([3, 3])
#Droom = np.array([4, 4])
#Aroom = np.array([2, 1])
#Broom = np.array([3, 4])
#Croom = np.array([2, 3])
#Droom = np.array([4, 1])

#D#C#B#A#
#D#B#A#C#
rooms = np.array([Aroom, Broom, Croom, Droom])
print(rooms)

np.all([np.all(rooms[x-1,:] == x) for x in range(1,5)])
#for x in range(1,5):
#    print(np.all(rooms[x-1,:] == x))

[ 0  0 -1  0 -1  0 -1  0 -1  0  0]
[[4 4 4 2]
 [4 3 2 1]
 [3 2 1 2]
 [3 1 3 1]]


False

In [41]:
minScore = 999999999999
def recurse(hallway, rooms, score):
    global minScore
    dests = []
    if np.all([np.all(rooms[x-1,:] == x) for x in range(1,5)]):
        if score < minScore:
            minScore = score
            print(f"score: {minScore}")
    #print("---rooms---")
    for i, room in enumerate(rooms):
        for j, val in enumerate(room):
            if (np.all(room[:j] == 0) or j == 0) and not val == 0:
                if val == i+1 and np.all(room[j+1:5] == i+1):
                    continue                 
                dests.append([(i,j), destinations((val, i), hallway, rooms, j+1)])
    #print("---Halways---")
    for i, x in enumerate(hallway):
        if x > 0:
            d3 = destinations((x, i), hallway, rooms, 0)
            if d3:
                dests.append([(-1,i), d3])
    #return dests
    #print(dests)
    for s_pos, dests_l in dests:
        #print(s_pos, dests_l)
        for l in dests_l:
            e_pos, steps = l
            nhallway = hallway.copy()
            nrooms = rooms.copy()
                
            if s_pos[0] == -1:
                val = nhallway[s_pos[1]]
                nhallway[s_pos[1]] = 0
            elif s_pos[0] > -1:
                val = nrooms[s_pos[0]][s_pos[1]]
                nrooms[s_pos[0]][s_pos[1]] = 0
                
            if e_pos[0] == -1:
                nhallway[e_pos[1]] = val
            elif e_pos[0] > -1:
                nrooms[e_pos[0]][e_pos[1]] = val
            recurse(nhallway, nrooms, score + (steps * move_num[val]))
                         
            
# -1: hallway
# 0 : Aroom
# 1: Broom
# 2: Croom
# 3: Droom
def destinations(pos, hallway, rooms, steps):
    # pos = (value, position/room)
    if steps > 0:
        startLocation = exits[pos[1]]
    else:
        startLocation = pos[1]
    destinations = []
    accessible = set()
    for x in range(startLocation+1, 11): #+1
        if not hallway[x] > 0:
            accessible.add(x)
        else:
            break
    for x in range(startLocation-1, -1, -1): #-1
        if not hallway[x] > 0:
            accessible.add(x)
        else:
            break
    if exits[pos[0]-1] in accessible:
        if np.all(rooms[pos[0]-1][np.where(rooms[pos[0]-1] != 0)] == pos[0]) or np.all(rooms[pos[0]-1] == 0):
            newPos = np.where(rooms[pos[0]-1] == 0)[0][-1]
            destinations.append([(pos[0]-1, newPos), steps + abs(startLocation - exits[pos[0]-1]) + newPos+1])
            return destinations
    if steps > 0:
        accessible.difference_update({2,4,6,8})
        for x in accessible:
            destinations.append([(-1,x), steps + abs(startLocation - x)])
    return destinations    

[(0, 373437429), (1, 378937429), (423, 373437429), (593, 373437429)]


In [42]:
recurse(hallway, rooms, 0)

score: 44519
score: 44117
score: 44115
score: 43919
score: 43917
score: 43317
score: 43119
score: 43117


In [31]:
min(scores)

NameError: name 'scores' is not defined

In [129]:
room = np.array([0,3,1,1])
for j, val in enumerate(room):
    print((np.all(room[:j] == 0) or j == 0) and not val == 0)
    #print((np.all(room[:j] == 0)))

False
True
False
False


In [88]:
np.all(room[:3] == 0)

False

In [89]:
room[:1]

array([1])

In [90]:
room

array([1, 4, 4, 4])

In [93]:
np.all(room[0+1:5] == 4)

True

In [97]:
4 in room

True

In [117]:
np.all(room[np.where(room != 0)] == 4) or np.all(room == 0)

False

In [103]:
room[np.where(room != 0)]

array([4, 4])

In [130]:
np.where(room == 0)[0][-1]

0

In [ ]:
destinations(pos, hallway, rooms, steps)
destinations(pos, hallway, rooms, steps)